<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import re
import gc
import random
import os

from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Config

In [108]:
n_splits = 5
input_path = '/content/drive/MyDrive/ColabNotebooks/QIQC/data'
max_features = 120000 # None
embed_size = 300
maxlen = 70 # None
batch_size = 512
max_epoch = 5

Load train and test data

In [36]:
train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

Tokenize

In [37]:
to_exclude = ''
to_tokenize = puncts

tokenizer = Tokenizer(filters=to_exclude, lower=False)

In [38]:
train_text = train_df["question_text"].fillna("_na_")
test_text = test_df["question_text"].fillna("_na_")
train_y = train_df['target'].values

In [39]:
train_text = train_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
test_text = test_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values

In [40]:
text = list(train_text) + list(test_text)

In [41]:
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index

In [109]:
train_X = tokenizer.texts_to_sequences(train_text)
test_X = tokenizer.texts_to_sequences(test_text)

train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [110]:
max_features = len(word_index) if max_features is None else max_features
# maxlen = max({len(seq) for seq in train_X} | {len(seq) for seq in val_X} | {len(seq) for seq in test_X}) if maxlen is None else maxlen

Load embedding

In [111]:
import zipfile
from gensim.models import KeyedVectors

embeddings_path = os.path.join(input_path, 'embeddings.zip')

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
para = 'paragram_300_sl999/paragram_300_sl999.txt'


def load_embedding(embedding_name):

    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

    with zipfile.ZipFile(embeddings_path) as embeddings_zip:

        if embedding_name == google:
            return KeyedVectors.load_word2vec_format(embeddings_zip.open(google), binary=True)

        else:
            embedding = []
            for o in embeddings_zip.open(embedding_name):
                try:
                    if len(o.decode('utf-8')) > 100:
                        embedding.append(get_coefs(*o.decode('utf-8').split(" ")))
                except:
                    pass
        
        return dict(embedding)

In [112]:
import nltk
# nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
s = PorterStemmer()
l = LancasterStemmer()
n = WordNetLemmatizer()

def is_in_emb(word, i, embedding, emb_matrix):

    if word in embedding:
        emb_matrix[i] = embedding[word]
        return True
    
    tmp = word.lower()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = word.upper()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = word.capitalize()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = s.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = l.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = n.lemmatize(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    return False

In [113]:
puncts = '´‘’“”…!#$%&()*+,-./:;<=>?@[\]^_`{|}~"' + "'"

In [114]:
def load_emb_matrix(embedding_name):

    embedding = load_embedding(embedding_name)
    if embedding_name == glove: emb_mean, emb_std = -0.005838499, 0.48782197
    if embedding_name == wiki: emb_mean, emb_std = -0.0033469985, 0.109855495
    if embedding_name == google: emb_mean, emb_std = -0.003527845, 0.13315111
    if embedding_name == para: emb_mean, emb_std = -0.0053248387, 0.49346521
    emb_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in tqdm(word_index.items()):
        i -= 1
        assert i >= 0

        if i >= max_features: continue

        if is_in_emb(word, i, embedding, emb_matrix): continue

        if embedding_name == google:

            tmp = re.sub('[0-9]{5,}', '#####', word)
            tmp = re.sub('[0-9]{4}', '####', tmp)
            tmp = re.sub('[0-9]{3}', '###', tmp)
            tmp = re.sub('[0-9]{2}', '##', tmp)
            if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

            for punct in puncts:
                tmp = tmp.replace(punct, '')
            if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

        else:
        
            tmp = word
            for punct in puncts:
                tmp = tmp.replace(punct, '')
            if is_in_emb(tmp, i, embedding, emb_matrix): continue

            for num in '0123456789':
                tmp = tmp.replace(num, '')
            if is_in_emb(tmp, i, embedding, emb_matrix): continue
    

    del embedding
    gc.collect()
    return emb_matrix

In [115]:
%%time
glove_emb_matrix = load_emb_matrix(glove)


CPU times: user 4min 3s, sys: 6.7 s, total: 4min 10s
Wall time: 4min 9s


In [116]:
%%time
para_emb_matrix = load_emb_matrix(para)


CPU times: user 3min 16s, sys: 4.28 s, total: 3min 20s
Wall time: 3min 19s


In [117]:
emb_matrix = np.mean((0.7 * glove_emb_matrix, 0.3 * para_emb_matrix), axis=0)

Training

https://www.kaggle.com/dohyun141/bilstm-pytorch-and-keras/edit

In [126]:
def train_model(model, train_X=train_X, train_y=train_y, n_splits=n_splits):
    splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(train_X, train_y))
    train_y = np.array(train_y)

    train_oof_preds = np.zeros((train_X.shape[0]))
    for i, (train_idx, val_idx) in enumerate(splits):
        print(f'Fold {i + 1}')

        train_X_fold = train_X[train_idx.astype(int)]
        train_y_fold = train_y[train_idx.astype(int)]
        val_X_fold = train_X[val_idx.astype(int)]
        val_y_fold = train_y[val_idx.astype(int)]

        clf = copy.deepcopy(model)
        clf.fit(train_X_fold, train_y_fold, batch_size=batch_size, epochs=max_epoch, validation_data=(val_X_fold, val_y_fold))
        
        valid_preds_fold = clf.predict(val_X_fold)[:,0]

        train_oof_preds[valid_idx] = valid_preds_fold
  
    return train_oof_preds

In [132]:
def make_model(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable=False)(inp)

    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [153]:
train_X[100]

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      3,     14,   7363,     12,   7208,
           12,    283,   1938,     11,   7261,     78,      8,   2492,
        40111,     29, 149435,   5458,    146,      1], dtype=int32)

In [139]:
len(train_y)

1306122

In [142]:
emb_matrix.shape

(120000, 300)

In [144]:
model = make_model(emb_matrix)

In [154]:
SEED=1

In [158]:
def model_train_cv(x_train,y_train,nfold,model_obj):
    splits = list(StratifiedKFold(n_splits=nfold, shuffle=True, random_state=SEED).split(x_train, y_train))
    x_train = x_train
    y_train = np.array(y_train)
    # matrix for the out-of-fold predictions
    train_oof_preds = np.zeros((x_train.shape[0]))
    for i, (train_idx, valid_idx) in enumerate(splits):
        print(f'Fold {i + 1}')
        x_train_fold = x_train[train_idx.astype(int)]
        y_train_fold = y_train[train_idx.astype(int)]
        x_val_fold = x_train[valid_idx.astype(int)]
        y_val_fold = y_train[valid_idx.astype(int)]

        clf = copy.deepcopy(model_obj)
        clf.fit(x_train_fold, y_train_fold, batch_size=512, epochs=5, validation_data=(x_val_fold, y_val_fold))
        
        valid_preds_fold = clf.predict(x_val_fold)[:,0]

        # storing OOF predictions
        train_oof_preds[valid_idx] = valid_preds_fold
    return train_oof_preds


In [159]:
train_oof_preds = model_train_cv(train_X[:8000], train_y[:8000], 5, model)

Fold 1
Epoch 1/5


AssertionError: ignored

In [134]:
train_oof_preds = train_model(model)

Fold 1
Epoch 1/5


AssertionError: ignored

In [157]:
!pip install tensorflow==2.0.0b1

     |████████████████████████████████| 88.7MB 59kB/s 
     |████████████████████████████████| 501kB 45.2MB/s 
     |████████████████████████████████| 3.1MB 41.7MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 2.0.0b1 which is incompatible.
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [94]:
pad_sequences(train_X).shape[0]

1306122

In [96]:
train_y

array([0, 0, 0, ..., 0, 0, 0])

In [97]:
np.zeros(100)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [98]:
np.zeros((100))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [79]:
len(a)

1044897

In [80]:
b

array(['Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved , completely disregarding their feelings /lives so you get to have something go your way and feel temporarily at ease . How did things change ?',
       'How were the Calgary Flames founded ?',
       'I am 30 , living at home and have no boyfriend . I would love a boyfriend and my own home . How can I progress my situation ?',
       ..., 'How is DSATM civil engineering ?',
       'What is your story today ?',
       'What other technical skills do you need as a computer science undergrad other than c and c + + ?'],
      dtype=object)

In [81]:
len(b)

261225

references:

https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go

https://www.kaggle.com/alhalimi/tokenization-and-word-embedding-compatibility

https://www.kaggle.com/canming/ensemble-mean-iii-64-36

https://www.kaggle.com/tks0123456789/pme-ema-6-x-8-epochs

https://www.kaggle.com/bkkaggle/pytorch-determinism-test

https://www.kaggle.com/hsinwenchang/more-bidirectional-cudnnlstm-layer

Please verify your config values.

In [10]:
seed = 31
n_splits = 5
input_path = '/content/drive/MyDrive/ColabNotebooks/QIQC/data'

In [11]:
def seed_torch(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [12]:
import gc

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
import os

train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [14]:
if os.path.exists(train_path): train_df = pd.read_csv(train_path)
else: train_df = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test_df = pd.read_csv(train_path)
else: test_df = pd.read_csv(test_path + '.zip')

print('train shape:', train_df.shape)
print('test shape:', test_df.shape)

KeyboardInterrupt: ignored

In [ ]:
import zipfile
from gensim.models import KeyedVectors

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
para = 'paragram_300_sl999/paragram_300_sl999.txt'


def load_embedding(embedding_name):

    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

    with zipfile.ZipFile(embeddings_path) as embeddings_zip:
        print("Found embeddings as a zip file")

        if embedding_name == google:
            return KeyedVectors.load_word2vec_format(embeddings_zip.open(google), binary=True)

        else:
            embedding = []
            for o in embeddings_zip.open(embedding_name):
                try:
                    if len(o.decode('utf-8')) > 100:
                        embedding.append(get_coefs(*o.decode('utf-8').split(" ")))
                except:
                    pass
        
        return dict(embedding)

In [ ]:
%%time
glove_embedding = load_embedding(glove)

Some features about glove_embedding

In [ ]:
"'re" in glove_embedding

In [ ]:
"you're" in glove_embedding

In [ ]:
"12345" in glove_embedding

In [ ]:
"'m" in glove_embedding

In [ ]:
for punct in '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~‘’“”…' + "'":
    if punct not in glove_embedding: print(punct)

We use all tokens from both train and test data for our vocabulary.

Tokenizer : keras

In [ ]:
puncts = '´‘’“”…!#$%&()*+,-./:;<=>?@[\]^_`{|}~"' + "'"

In [ ]:
from keras.preprocessing.text import Tokenizer

to_exclude = ''
to_tokenize = puncts

tokenizer = Tokenizer(filters=to_exclude, lower=False)

In [ ]:
from sklearn.model_selection import train_test_split, KFold

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

train_df, val_df = train_test_split(train_df, test_size=0.1)

train_idx, val_idx = list(kfold.split(train_df))[0]
train_df, val_df = train_df.iloc[train_idx], train_df.iloc[val_idx]

train_text = train_df["question_text"].fillna("_na_")
val_text = val_df["question_text"].fillna("_na_")
test_text = test_df["question_text"].fillna("_na_")

train_y = train_df['target'].values
val_y = val_df['target'].values

In [ ]:
import re

train_text = train_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
val_text = val_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
test_text = test_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values

In [ ]:
text = list(train_text) + list(val_text) + list(test_text)

In [ ]:
%%time
tokenizer.fit_on_texts(text)

In [ ]:
word_index = tokenizer.word_index

In [ ]:
%%time
train_X = tokenizer.texts_to_sequences(train_text)
val_X = tokenizer.texts_to_sequences(val_text)
test_X = tokenizer.texts_to_sequences(test_text)

maxlen = max({len(seq) for seq in train_X} | {len(seq) for seq in val_X} | {len(seq) for seq in test_X})

In [ ]:
from keras.preprocessing.sequence import pad_sequences

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [ ]:
glove_emb_mean, glove_emb_std = -0.005838499, 0.48782197

In [ ]:
embed_size = 300 # how big is each word vector
max_features = len(word_index) # how many unique words to use (i.e num rows in embedding vector)

Without any effort

In [ ]:
show_inital_emb_matrix = True # set False if you do not want to see the intial embedding matrix

In [ ]:
if show_inital_emb_matrix:
    glove_emb_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
    glove_oov = {}
    for word, i in tqdm(word_index.items()):
        if word in glove_embedding:
            glove_emb_matrix[i] = glove_embedding[word]
        
        else:
            glove_oov[word] = id

    print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))

else:
    print('percentage of oov of glove: 41.39%')

Let's make an improved embedding matrix.

In [ ]:
import nltk
# nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
s = PorterStemmer()
l = LancasterStemmer()
n = WordNetLemmatizer()

In [ ]:
def is_in_emb(word, i, embedding, emb_matrix):

    if word in embedding:
        emb_matrix[i] = embedding[word]
        return True
    
    tmp = word.lower()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = word.upper()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = word.capitalize()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = s.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = l.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = n.lemmatize(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    return False

In [ ]:
glove_emb_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
glove_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, glove_embedding, glove_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, glove_embedding, glove_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, glove_embedding, glove_emb_matrix): continue

    else:
        glove_oov[word] = i

print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))

In [ ]:
del glove_embedding
gc.collect()

41.39% -> 14.11%

Wiki

In [ ]:
%%time
wiki_embedding = load_embedding(wiki)

In [ ]:
for punct in '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~‘’“”…' + "'":
    if punct not in wiki_embedding: print(punct)

In [ ]:
"3123" in wiki_embedding

In [ ]:
"I'm" in wiki_embedding

In [ ]:
"'m" in wiki_embedding

In [ ]:
wiki_emb_mean, wiki_emb_std = -0.0033469985, 0.109855495

In [ ]:
if show_inital_emb_matrix:
    wiki_emb_matrix = np.random.normal(wiki_emb_mean, wiki_emb_std, (max_features, embed_size))
    wiki_oov = {}
    for word, i in tqdm(word_index.items()):
        if word in wiki_embedding:
            wiki_emb_matrix[i] = wiki_embedding[word]
        
        else:
            wiki_oov[word] = id

    print('percentage of oov of wiki: {:.2f}%'.format(len(wiki_oov) / max_features * 100))

else:
    print('percentage of oov of wiki: 46.91%')

In [ ]:
wiki_emb_matrix = np.random.normal(wiki_emb_mean, wiki_emb_std, (max_features, embed_size))
wiki_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, wiki_embedding, wiki_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, wiki_embedding, wiki_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, wiki_embedding, wiki_emb_matrix): continue

    else:
        wiki_oov[word] = i

print('percentage of oov of wiki: {:.2f}%'.format(len(wiki_oov) / max_features * 100))

In [ ]:
del wiki_embedding
gc.collect()

google

In [ ]:
%%time
google_embedding = load_embedding(google)

In [ ]:
'?' in google_embedding

In [ ]:
google_emb_mean, google_emb_std = -0.003527845, 0.13315111

In [ ]:
google_emb_matrix = np.random.normal(google_emb_mean, google_emb_std, (max_features, embed_size))
google_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, google_embedding, google_emb_matrix): continue

    tmp = re.sub('[0-9]{5,}', '#####', word)
    tmp = re.sub('[0-9]{4}', '####', tmp)
    tmp = re.sub('[0-9]{3}', '###', tmp)
    tmp = re.sub('[0-9]{2}', '##', tmp)
    if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

    # tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

    else:
        google_oov[word] = i

print('percentage of oov of google: {:.2f}%'.format(len(google_oov) / max_features * 100))

In [ ]:
google_oov

In [ ]:
del google_embedding
gc.collect()

para

In [ ]:
%%time
para_embedding = load_embedding(para)

In [ ]:
para_emb_mean, para_emb_std = -0.0053248387, 0.49346521

In [ ]:
para_emb_matrix = np.random.normal(para_emb_mean, para_emb_std, (max_features, embed_size))
para_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, para_embedding, para_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, para_embedding, para_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, para_embedding, para_emb_matrix): continue

    else:
        para_oov[word] = i

print('percentage of oov of para: {:.2f}%'.format(len(para_oov) / max_features * 100))

In [ ]:
para_oov

In [ ]:
del para_embedding
gc.collect()